In [14]:
from baseline_transformer_POS_unk import *
import copy

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())  # Renvoie True si un GPU est disponible

batch_size=16
epochs=100



True


In [16]:
#load data
sentences, pos_tags = load_data("UD_French-Sequoia/fr_sequoia-ud-train.conllu")

#init le vocab
word_counts = Counter(word for sentence in sentences for word in sentence)
word_to_ix = {word: i+1 for i, word in enumerate(word_counts)}  # +1 pour le padding
word_to_ix['<UNK>'] = 1

tag_counts = Counter(tag for tags in pos_tags for tag in tags)
tag_to_ix = {tag: i for i, tag in enumerate(tag_counts)}

#params
embedding_dim = 512
nhead = 4
nhid = 1024
nlayers = 3
batch_size = 16
epochs = 100



#init data et dataloader
dataset = POSDataset(sentences, pos_tags, word_to_ix, tag_to_ix)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
print(len(dataset))
print(f"nombre de batchs : {len(data_loader)}")
#init transformer, loss et optimizer
#changer les parametres en CV pour optimiser tout ça
model = POSTransformerModel(len(word_to_ix), embedding_dim, nhead, nhid, nlayers, len(tag_to_ix))
loss_function = nn.CrossEntropyLoss(ignore_index=-1)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

validation_sentences, validation_pos_tags = load_data("UD_French-Sequoia/fr_sequoia-ud-dev.conllu")
# Préparer le DataLoader pour les données de validation
validation_dataset = POSDataset(validation_sentences, validation_pos_tags, word_to_ix, tag_to_ix)
validation_data_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)







2231
nombre de batchs : 140


C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [17]:

# Paramètres pour l'arrêt précoce
patience = 5  # Nombre d'époques à attendre après la dernière amélioration de la loss de validation
best_val_accuracy = 0
epochs_no_improve = 0
# Assuming tag_counts is a Counter object containing counts of each tag
print(tag_counts)
most_common_tag = tag_counts.most_common(1)[0][0]
print(most_common_tag)

#Training
model.to(device)  # Déplacer le modèle sur le GPU si disponible
for epoch in range(epochs): 
    model.train()
    total_loss = 0
    for sentence_in, targets in data_loader:
        sentence_in, targets = sentence_in.to(device), targets.to(device)  # Déplacer les données sur le périphérique
        optimizer.zero_grad()
        tag_scores = model(sentence_in)
        loss = loss_function(tag_scores.view(-1, len(tag_to_ix)), targets.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Utiliser la fonction modifiée pour évaluer la validation loss et l'accuracy
    val_loss, val_accuracy = evaluate_model(model, validation_data_loader, loss_function,tag_to_ix)
    print(f"Epoch {epoch+1}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_model = copy.deepcopy(model)
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    # Arrêt précoce si aucune amélioration
    if epochs_no_improve == patience:
        print("Arrêt précoce : La loss de validation ne s'améliore plus")
        break
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(data_loader)}")

# Modifier la section de prédiction pour déplacer les entrées sur le périphérique
model.eval()
with torch.no_grad():
    inputs = torch.tensor([word_to_ix[word] for word in sentences[0]], dtype=torch.long).unsqueeze(0).to(device)
    tag_scores = model(inputs)
    predicted_tags = [list(tag_to_ix.keys())[tag] for tag in tag_scores[0].argmax(dim=1).cpu()]
    print(f"Sentence: {' '.join(sentences[0])}")
    print(f"Predicted POS Tags: {predicted_tags}")
    true_tags = [tag for tag in pos_tags[0]]
    print(f"Vraies étiquettes POS: {true_tags}")




# Charger les données de test
test_sentences, test_pos_tags = load_data("UD_French-Sequoia/fr_sequoia-ud-test.conllu")
# Prepare the test dataset
unk_count_test=0
nombre_mot=0
test_dataset_sentences = []
for sentence in test_sentences:
    indexed_sentence = []
    for word in sentence:
        nombre_mot+=1
        if word in word_to_ix:
            indexed_sentence.append(word_to_ix[word])
        else:
            indexed_sentence.append(word_to_ix['<UNK>'])
            unk_count_test += 1  # Increment the counter for each unknown word
    test_dataset_sentences.append(torch.tensor(indexed_sentence, dtype=torch.long))

# Préparer le DataLoader pour les données de test
test_dataset = POSDataset(test_sentences, test_pos_tags, word_to_ix, tag_to_ix)
test_data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Calculer l'accuracy
loss,accuracy = evaluate_model(model,test_data_loader,loss_function,tag_to_ix)
print(f"Test Accuracy : {accuracy:.4f}")


Counter({'NOUN': 10870, 'ADP': 8043, 'DET': 7439, 'PUNCT': 5702, 'VERB': 4248, 'ADJ': 3126, 'PROPN': 2410, 'PRON': 2083, 'ADV': 1890, 'AUX': 1623, '_': 1360, 'CCONJ': 1202, 'NUM': 1183, 'SCONJ': 550, 'X': 127, 'SYM': 15, 'INTJ': 3, 'PART': 3})
NOUN
Epoch 1, Validation Loss: 0.7359, Validation Accuracy: 0.7612
Epoch 1, Loss: 1.1762406400271825
Epoch 2, Validation Loss: 0.6043, Validation Accuracy: 0.8073
Epoch 2, Loss: 0.7137869298458099
Epoch 3, Validation Loss: 0.5453, Validation Accuracy: 0.8229
Epoch 3, Loss: 0.5902704879641533
Epoch 4, Validation Loss: 0.4875, Validation Accuracy: 0.8426
Epoch 4, Loss: 0.5126934545380729
Epoch 5, Validation Loss: 0.4616, Validation Accuracy: 0.8643
Epoch 5, Loss: 0.4408868016941207
Epoch 6, Validation Loss: 0.4306, Validation Accuracy: 0.8726
Epoch 6, Loss: 0.38171730147940774
Epoch 7, Validation Loss: 0.3890, Validation Accuracy: 0.8805
Epoch 7, Loss: 0.3301039992698601
Epoch 8, Validation Loss: 0.4214, Validation Accuracy: 0.8926
Epoch 8, Loss: 0

In [18]:
# Modifier la section de prédiction pour déplacer les entrées sur le périphérique
model.eval()
with torch.no_grad():
    inputs = torch.tensor([word_to_ix[word] for word in sentences[0]], dtype=torch.long).unsqueeze(0).to(device)
    tag_scores = model(inputs)
    predicted_tags = [list(tag_to_ix.keys())[tag] for tag in tag_scores[0].argmax(dim=1).cpu()]
    print(f"Sentence: {' '.join(sentences[0])}")
    print(f"Predicted POS Tags: {predicted_tags}")
    true_tags = [tag for tag in pos_tags[0]]
    print(f"Vraies étiquettes POS: {true_tags}")




# Charger les données de test
test_sentences, test_pos_tags = load_data("UD_French-Sequoia/fr_sequoia-ud-test.conllu")
# Prepare the test dataset
unk_count_test=0
nombre_mot=0
test_dataset_sentences = []
for sentence in test_sentences:
    indexed_sentence = []
    for word in sentence:
        nombre_mot+=1
        if word in word_to_ix:
            indexed_sentence.append(word_to_ix[word])
        else:
            indexed_sentence.append(word_to_ix['<UNK>'])
            unk_count_test += 1  # Increment the counter for each unknown word
    test_dataset_sentences.append(torch.tensor(indexed_sentence, dtype=torch.long))

# Préparer le DataLoader pour les données de test
test_dataset = POSDataset(test_sentences, test_pos_tags, word_to_ix, tag_to_ix)
test_data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Calculer l'accuracy
loss,accuracy = evaluate_model(model,test_data_loader,loss_function,tag_to_ix)
print(f"Test Accuracy : {accuracy:.4f}")

Sentence: gutenberg
Predicted POS Tags: ['PROPN']
Vraies étiquettes POS: ['PROPN']
Test Accuracy : 0.8916


In [19]:
# Modifier la section de prédiction pour déplacer les entrées sur le périphérique
best_model.eval()
with torch.no_grad():
    inputs = torch.tensor([word_to_ix[word] for word in sentences[0]], dtype=torch.long).unsqueeze(0).to(device)
    tag_scores = best_model(inputs)
    predicted_tags = [list(tag_to_ix.keys())[tag] for tag in tag_scores[0].argmax(dim=1).cpu()]
    print(f"Sentence: {' '.join(sentences[0])}")
    print(f"Predicted POS Tags: {predicted_tags}")
    true_tags = [tag for tag in pos_tags[0]]
    print(f"Vraies étiquettes POS: {true_tags}")




# Charger les données de test
test_sentences, test_pos_tags = load_data("UD_French-Sequoia/fr_sequoia-ud-test.conllu")
# Prepare the test dataset
unk_count_test=0
nombre_mot=0
test_dataset_sentences = []
for sentence in test_sentences:
    indexed_sentence = []
    for word in sentence:
        nombre_mot+=1
        if word in word_to_ix:
            indexed_sentence.append(word_to_ix[word])
        else:
            indexed_sentence.append(word_to_ix['<UNK>'])
            unk_count_test += 1  # Increment the counter for each unknown word
    test_dataset_sentences.append(torch.tensor(indexed_sentence, dtype=torch.long))

# Préparer le DataLoader pour les données de test
test_dataset = POSDataset(test_sentences, test_pos_tags, word_to_ix, tag_to_ix)
test_data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Calculer l'accuracy
loss,accuracy = evaluate_model(best_model,test_data_loader,loss_function,tag_to_ix)
print(f"Test Accuracy : {accuracy:.4f}")

Sentence: gutenberg
Predicted POS Tags: ['VERB']
Vraies étiquettes POS: ['PROPN']
Test Accuracy : 0.8955
